# MyAnimeList Database 2020  
https://www.kaggle.com/hernan4444/anime-recommendation-database-2020?select=animelist.csv  
- fm에서는 개인별로 anime rating을 예측  

325772 users, 17562 animes  

중요 columns:  
- user_id  
- anime_id(MAL_ID)    
- rating(score): user 별로 1-10 매긴다.  
- anime name(English name)
- genres(anime 별 장르) 
등

---



## 1. 데이터 불러오기
데이터 준비

In [ ]:
# import
import numpy as np
import pandas as pd
import random

import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [ ]:
# GPU 확인
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
# 자료형 선언
tf.keras.backend.set_floatx('float32')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
anime_list = pd.read_csv('/content/drive/Shareddrives/DSL_2021_2_추천시스템/추천시스템 발표구현/Wide & Deep/anime_merge_sampling_100000.csv')
# print(len(anime_list[anime_list['rating'] == 0]))   # 0
# print(len(anime_list[anime_list['rating'] != 0]))   # 100000

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (31,36,37,38,40) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
anime_list.head(3)

,Unnamed: 0,user_id,anime_id,rating,watching_status,watched_episodes,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,Producers,Licensors,Studios,Source,Duration,Rating,Ranked,Popularity,Members,Favorites,Watching,Completed,On-Hold,Dropped,Plan to Watch,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
0,32752453,186051,18679,7,2,24,18679,Kill la Kill,8.11,"Action, Comedy, Super Power, Ecchi, School",KILL la KILL,キルラキル,TV,24,"Oct 4, 2013 to Mar 28, 2014",Fall 2013,"Aniplex, Dentsu, Kadokawa Shoten, Mainichi Bro...",Aniplex of America,Trigger,Original,24 min. per ep.,PG-13 - Teens 13 or older,402.0,35,1283021,34658,88881,838228,62391,67845,225676,109240,176583.0,211585.0,131387.0,50735.0,21899.0,10849.0,4244.0,2495.0,2557.0
1,20019520,113524,12531,9,2,12,12531,Sakamichi no Apollon,8.36,"Drama, Josei, Music, Romance, School",Kids on the Slope,坂道のアポロン,TV,12,"Apr 13, 2012 to Jun 29, 2012",Spring 2012,"Dentsu, Fuji TV, Toho, DAX Production, Sony Mu...",Sentai Filmworks,"Tezuka Productions, MAPPA",Manga,22 min. per ep.,PG-13 - Teens 13 or older,176.0,494,284901,5914,11918,159591,7785,5322,100285,25513,40867.0,39221.0,19232.0,6288.0,2411.0,864.0,317.0,149.0,383.0
2,30964830,175828,4224,9,2,25,4224,Toradora!,8.24,"Slice of Life, Comedy, Romance, School",Toradora!,とらドラ！,TV,25,"Oct 2, 2008 to Mar 26, 2009",Fall 2008,"Genco, Starchild Records, Magic Capsule, Yomiu...","NIS America, Inc.",J.C.Staff,Light novel,24 min. per ep.,PG-13 - Teens 13 or older,265.0,17,1567792,50708,72442,1191775,42141,41098,220336,185286,255249.0,273494.0,160797.0,58749.0,25479.0,10810.0,4372.0,2456.0,2890.0


In [ ]:
anime_list.shape

(100000, 41)

In [ ]:
anime_list = anime_list[['user_id', 'anime_id', 'rating', 'watching_status', 'watched_episodes', 
                         'Genres', 'Type', 'Episodes', 'Aired', 'Source', 'Rating', 'Members']]
anime_list[anime_list == 'Unknown'] = np.nan #Unknown값 nan으로 대체 후 dropna 실행
anime_list.dropna(axis=0, inplace=True) #제거 후 데이터 크기 : (97949, 12)
anime_list.shape

(97949, 12)

In [ ]:
len(anime_list['user_id'].unique())

71573

In [ ]:
anime_list.dtypes

user_id              int64
anime_id             int64
rating               int64
watching_status      int64
watched_episodes     int64
Genres              object
Type                object
Episodes            object
Aired               object
Source              object
Rating              object
Members              int64
dtype: object

In [ ]:
anime_list['Episodes'] = anime_list['Episodes'].astype('int32')

In [ ]:
anime_list['episode_rate'] = anime_list['watched_episodes'] / anime_list['Episodes']

In [ ]:
anime_list.sample()

,user_id,anime_id,rating,watching_status,watched_episodes,Genres,Type,Episodes,Aired,Source,Rating,Members,episode_rate
19154,42612,5112,3,3,8,"Comedy, Ecchi, Fantasy, Romance",TV,12,"Jan 4, 2009 to Mar 29, 2009",Light novel,PG-13 - Teens 13 or older,109315,0.666667


In [ ]:
# 71573명 중 10000명의 user을 임의로 선정하여 추천 시스템 만들기
# 71573명의 user로 fm model을 돌릴 수가 없음.
random.seed(1)
user10000_list = random.sample(list(anime_list['user_id'].unique()), 10000)

In [ ]:
anime_list = anime_list[anime_list['user_id'].apply(lambda x: x in user10000_list)]
anime_list.shape

(13769, 13)

In [ ]:
anime_list1 = anime_list[['user_id', 'anime_id', 'rating', 'watching_status', 'episode_rate']]
print(anime_list1.shape)

(13769, 5)


In [ ]:
anime = anime_list[['anime_id', 'Genres', 'Type', 'Aired', 'Source', 'Rating', 'Members']]
anime = anime.drop_duplicates(['anime_id'])
anime.shape

(3802, 7)

## column 별로 처리

- Type: 별다른 처리가 필요하지 않음.

In [ ]:
# Type: 'TV', 'Movie', 'OVA', 'Special', 'ONA', 'Music'
anime['Type'].value_counts()

TV         1964
OVA         700
Movie       507
Special     418
ONA         162
Music        51
Name: Type, dtype: int64

- Source: 개수가 적은 카테고리를 Other에 합칠까 고민했지만 그냥 지금 상태 유지하기로 했음. -> 별다른 처리 X

In [ ]:
anime['Source'].value_counts()

Manga            1599
Original          782
Light novel       490
Visual novel      280
Game              197
4-koma manga      121
Novel             108
Web manga          92
Other              80
Music              20
Card game          14
Book               13
Digital manga       4
Picture book        2
Name: Source, dtype: int64

- Rating -> 별다른 처리가 필요없음.

In [ ]:
anime['Rating'].value_counts()

PG-13 - Teens 13 or older         2218
R - 17+ (violence & profanity)     623
R+ - Mild Nudity                   395
G - All Ages                       250
Rx - Hentai                        180
PG - Children                      136
Name: Rating, dtype: int64

- Aired  
데이터셋 Aired 칼럼을 카테고리 타입으로 바꾸기  
10단위로 끊어서 생각하기

In [ ]:
anime['Aired'].isnull().sum()

0

In [ ]:
anime['Aired'] = anime['Aired'].str.extract('(\d\d\d\d)')

In [ ]:
anime['Aired'] = anime['Aired'].apply(lambda x: x[:3] + '0')
anime['Aired']

15       2010
31       2000
32       2010
33       2010
35       2000
         ... 
99928    2010
99932    2010
99952    2000
99971    2010
99986    2010
Name: Aired, Length: 3802, dtype: object

In [ ]:
anime['Aired'].value_counts()

2010    2180
2000    1119
1990     245
2020     128
1980      88
1970      21
1960      11
1930       6
1950       2
1940       1
1920       1
Name: Aired, dtype: int64

In [ ]:
anime['Aired'] = anime['Aired'].astype('int32')

In [ ]:
anime.loc[anime['Aired'] <= 1970, 'Aired'] = 'Below_1970s'

In [ ]:
anime['Aired'].value_counts()

2010           2180
2000           1119
1990            245
2020            128
1980             88
Below_1970s      42
Name: Aired, dtype: int64

- Members  
Members에 MinMaxscaler 적용: 1차원은 수행할 수 없음. -> 공식으로 적용

In [ ]:
s = anime['Members']
s = (s - s.min()) / (s.max() - s.min())
anime['Members'] = s

- Genres  
Genres는 따로 처리해주기

In [ ]:
anime1 = anime[['anime_id', 'Type', 'Aired', 'Source', 'Rating', 'Members']]

In [ ]:
# 장르 처리
genres_dummies = anime['Genres'].str.get_dummies(sep=', ')
anime2 = anime1.join(genres_dummies)
anime2.head()

,anime_id,Type,Aired,Source,Rating,Members,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,Harem,Hentai,Historical,Horror,Josei,Kids,Magic,Martial Arts,Mecha,Military,Music,Mystery,Parody,Police,Psychological,Romance,Samurai,School,Sci-Fi,Seinen,Shoujo,Shoujo Ai,Shounen,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri
15,7338,Special,2010,Original,R - 17+ (violence & profanity),0.084784,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
31,120,TV,2000,Manga,PG-13 - Teens 13 or older,0.162525,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0
32,36524,Special,2010,Visual novel,PG-13 - Teens 13 or older,0.008112,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
33,19367,TV,2010,Original,R - 17+ (violence & profanity),0.018971,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
35,4224,TV,2000,Light novel,PG-13 - Teens 13 or older,0.605420,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [ ]:
anime2.shape

(3802, 49)

## fm model의 input 형태 만들기

In [ ]:
merged = pd.merge(anime_list1, anime2, on = 'anime_id') # 장르 추가 O
# merged = pd.merge(anime_list1, anime1, on = 'anime_id') # 장르 추가 X
merged = merged.sort_values(by=['user_id', 'anime_id'])
merged.head(3)

,user_id,anime_id,rating,watching_status,episode_rate,Type,Aired,Source,Rating,Members,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,Harem,Hentai,Historical,Horror,Josei,Kids,Magic,Martial Arts,Mecha,Military,Music,Mystery,Parody,Police,Psychological,Romance,Samurai,School,Sci-Fi,Seinen,Shoujo,Shoujo Ai,Shounen,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri
6446,12,28121,10,2,1.000000,TV,2010,Light novel,PG-13 - Teens 13 or older,0.415152,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12920,19,15699,5,4,0.083333,TV,2010,Light novel,PG-13 - Teens 13 or older,0.057106,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1863,147,237,10,2,1.000000,TV,2000,Original,PG-13 - Teens 13 or older,0.146355,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
merged.shape

(13769, 53)

In [ ]:
merged['user_id'] = merged['user_id'].astype('category')
merged['anime_id'] = merged['anime_id'].astype('category')
merged['watching_status'] = merged['watching_status'].astype('category')
# Members, episode_rate는 숫자로 놔두기
merged['Aired'] = merged['Aired'].astype('category')
merged['Rating'] = merged['Rating'].astype('category')
merged['Type'] = merged['Type'].astype('category')
merged['Source'] = merged['Source'].astype('category')

In [ ]:
# 확인
len(merged['user_id'].unique()) + len(merged['anime_id'].unique()) + len(merged['watching_status'].unique()) + len(merged['Aired'].unique()) + len(merged['Rating'].unique()) + len(merged['Type'].unique()) + len(merged['Source'].unique()) + len(genres_dummies.columns) + 2

13884

In [ ]:
len(merged['user_id'].unique())

10000

In [ ]:
len(merged['anime_id'].unique())

3802

In [ ]:
input = pd.get_dummies(merged, sparse = True)
input

,rating,episode_rate,Members,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,Harem,Hentai,Historical,Horror,Josei,Kids,Magic,Martial Arts,Mecha,Military,Music,Mystery,Parody,Police,Psychological,Romance,Samurai,School,Sci-Fi,Seinen,Shoujo,Shoujo Ai,Shounen,Shounen Ai,Slice of Life,Space,Sports,...,anime_id_44087,anime_id_45598,anime_id_45614,watching_status_1,watching_status_2,watching_status_3,watching_status_4,watching_status_6,Type_Movie,Type_Music,Type_ONA,Type_OVA,Type_Special,Type_TV,Aired_1980,Aired_1990,Aired_2000,Aired_2010,Aired_2020,Aired_Below_1970s,Source_4-koma manga,Source_Book,Source_Card game,Source_Digital manga,Source_Game,Source_Light novel,Source_Manga,Source_Music,Source_Novel,Source_Original,Source_Other,Source_Picture book,Source_Visual novel,Source_Web manga,Rating_G - All Ages,Rating_PG - Children,Rating_PG-13 - Teens 13 or older,Rating_R - 17+ (violence & profanity),Rating_R+ - Mild Nudity,Rating_Rx - Hentai
6446,10,1.000000,0.415152,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
12920,5,0.083333,0.057106,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1863,10,1.000000,0.146355,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
4689,7,1.000000,0.227215,1,1,0,1,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
4235,7,1.000000,0.547431,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4122,5,1.000000,0.012971,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
8273,7,1.000000,0.111811,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
11571,7,1.000000,0.000965,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
7808,7,0.153846,0.012393,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0


In [ ]:
rating = input['rating']
input.drop('rating', axis = 1, inplace = True)

In [ ]:
input
# 아까 구했던 예상 column의 개수와 일치함.

,episode_rate,Members,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,Harem,Hentai,Historical,Horror,Josei,Kids,Magic,Martial Arts,Mecha,Military,Music,Mystery,Parody,Police,Psychological,Romance,Samurai,School,Sci-Fi,Seinen,Shoujo,Shoujo Ai,Shounen,Shounen Ai,Slice of Life,Space,Sports,Super Power,...,anime_id_44087,anime_id_45598,anime_id_45614,watching_status_1,watching_status_2,watching_status_3,watching_status_4,watching_status_6,Type_Movie,Type_Music,Type_ONA,Type_OVA,Type_Special,Type_TV,Aired_1980,Aired_1990,Aired_2000,Aired_2010,Aired_2020,Aired_Below_1970s,Source_4-koma manga,Source_Book,Source_Card game,Source_Digital manga,Source_Game,Source_Light novel,Source_Manga,Source_Music,Source_Novel,Source_Original,Source_Other,Source_Picture book,Source_Visual novel,Source_Web manga,Rating_G - All Ages,Rating_PG - Children,Rating_PG-13 - Teens 13 or older,Rating_R - 17+ (violence & profanity),Rating_R+ - Mild Nudity,Rating_Rx - Hentai
6446,1.000000,0.415152,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
12920,0.083333,0.057106,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1863,1.000000,0.146355,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
4689,1.000000,0.227215,1,1,0,1,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
4235,1.000000,0.547431,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4122,1.000000,0.012971,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
8273,1.000000,0.111811,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
11571,1.000000,0.000965,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
7808,0.153846,0.012393,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0


In [ ]:
X = input
Y = rating

In [ ]:
n, p = X.shape
# n: user-item pair 수
# p: indicator variable 수

In [ ]:
print(X.shape)
print(Y.shape)

(13769, 13884)
(13769,)


## 3. FM 모델 선언

In [ ]:
class FM(tf.keras.Model):
    def __init__(self, k):
        super(FM, self).__init__()

        # 모델의 파라미터 정의: initialize
        # 얘네를 SGD로 학습시킬 것이다. 
        self.w_0 = tf.Variable([0.0]) # w0: global bias
        self.w = tf.Variable(tf.zeros([p])) # w: 각 feature의 영향력을 모델화함.
        self.V = tf.Variable(tf.random.normal(shape=(p, k)))  

    def call(self, inputs):
        # 두번째 항
        # reduce_sum: 특정 차원을 제거하고 합계를 구함.(axis = 1: 가로축을 기준으로 합계를 구함.)
        linear_terms = tf.reduce_sum(tf.math.multiply(self.w, inputs), axis=1)

        # 세번째 항
        # 세번째 항을 어떻게 구현하느냐가 속도의 차이를 만듦. -> 위 그림처럼 구현시킴.
        # pow: 거듭제곱
        interactions = 0.5 * tf.reduce_sum(
            tf.math.pow(tf.matmul(inputs, self.V), 2)
            - tf.matmul(tf.math.pow(inputs, 2), tf.math.pow(self.V, 2)),
            1,
            keepdims=False
        ) #keepdims: shape와 관계없이 브로드캐스팅 가능하도록

        y_hat = self.w_0 + linear_terms + interactions

        return y_hat

In [ ]:
# Forward
# FM 모델에서의 inputs, targets에 따른 best parameter 찾아야 됨.
# best parameter 찾는 과정(optimizer 방식 정해야 함.) 
def train_on_batch(model, optimizer, inputs, targets):
    with tf.GradientTape() as tape: # 자동미분(연관된 연산을 tape에 기록)
        # y_pred(y_hat): self.w_0 + linear_terms + interactions
        y_pred = model(inputs)
        # loss: mse
        loss = tf.keras.losses.mean_squared_error(y_true=targets, y_pred=y_pred)
    
    # loss를 모델의 파라미터로 편미분하여 gradients를 구한다.
    grads = tape.gradient(target=loss, sources=model.trainable_variables)

    # apply_gradients()를 통해 processed gradients를 적용한다.
    # gradient는 optimizer에 의해 processed됨.
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    return loss


# 반복 학습 함수
# 여기서 num_batch는 batch의 개수를 말함.
def train(epochs, num_batch, learning_rate, k):
    # from_tensor_slices: It accepts single or multiple numpy arrays or tensors. Dataset created using this method will emit only one data at a time.
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify = Y, random_state=42)
    
    # train data
    train_ds = tf.data.Dataset.from_tensor_slices(
        (tf.cast(X_train, tf.float32), tf.cast(Y_train, tf.float32))).shuffle(500).batch(num_batch)

    # test data
    test_ds = tf.data.Dataset.from_tensor_slices(
        (tf.cast(X_test, tf.float32), tf.cast(Y_test, tf.float32))).shuffle(200).batch(num_batch)

    model = FM(k)
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    loss_history = []

    # 학습
    for i in range(epochs):
      for x, y in train_ds:
          loss = np.sqrt(train_on_batch(model, optimizer, x, y))
          loss_history.append(loss)

    #   if i % 2 == 0: # 2 간격으로 출력
    #       print("스텝 {:03d}에서 누적 평균 손실: {:.4f}".format(i, np.mean(loss_history)))

    # print(loss_history)

    # 테스트
    y_true_lst = []
    y_pred_lst = []
    test_loss_history = []
    for x, y in test_ds:
        y_pred = model(x)
        loss = np.sqrt(tf.keras.losses.mse(y_true=y, y_pred=y_pred))
        test_loss_history.append(loss)
        for true_value in y:
            y_true_lst.append(true_value)
        for pred_value in y_pred:
            y_pred_lst.append(pred_value)

    y_df = pd.DataFrame({'y_true': y_true_lst, 'y_pred': y_pred_lst})
    # print("테스트 평균 loss: {:.4f}".format(np.mean(test_loss_history)))

    return np.mean(test_loss_history), y_df

In [ ]:
# 현재까지의 best parameter
epochs = 30
num_batch = 8
learning_rate = 0.01
k = 3
loss, y_df = train(epochs, num_batch, learning_rate, k)
print("테스트 평균 loss: {:.4f}".format(loss))
print(y_df.sample(5))

테스트 평균 loss: 1.8597
                                        y_true                                         y_pred
2690  tf.Tensor(10.0, shape=(), dtype=float32)   tf.Tensor(9.352252, shape=(), dtype=float32)
1755   tf.Tensor(8.0, shape=(), dtype=float32)   tf.Tensor(7.106329, shape=(), dtype=float32)
1829  tf.Tensor(10.0, shape=(), dtype=float32)   tf.Tensor(7.335272, shape=(), dtype=float32)
411    tf.Tensor(7.0, shape=(), dtype=float32)    tf.Tensor(8.43277, shape=(), dtype=float32)
1511   tf.Tensor(8.0, shape=(), dtype=float32)  tf.Tensor(7.9647427, shape=(), dtype=float32)


best_params(epochs, batch_size, learning_rate, k):  (30, 8, 0.01, 3)  
loss: 1.7935786

In [ ]:
# k: number of latent factors
k = 3 # [3, 5]
num_batch_list = [8, 16, 32, 64, 128, 256, 512, 1024, 2048]
epochs = 30
learning_rate = 0.01
best_loss = 5
best_params = None


for num_batch in num_batch_list:
    loss = train(epochs, num_batch, learning_rate, k)
    
    if loss < best_loss:
        best_loss = loss
        best_params = (epochs, num_batch, learning_rate, k)

print("best loss: ", best_loss)
print("best_params(epochs, batch_size, learning_rate, k): ", best_params)